In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
FILENAME = 'NBADATA.csv'

In [3]:
def create_rolling_dataset(window,FILENAME,spread=False,winner=False):
    """Uses the dataframe to create rolling averages, in 
        an attempt to capture a team's performance over an N game window prior to the game. 
    """
    
    #retain relevant columns. 
    data = pd.read_csv(FILENAME) 
    data['3P%'] = np.divide(data['3P'].values,data['3PA'].values) 
    del data['3P'],data['3PA']
    data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
    del data['FG'],data['FGA']
    data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
    del data['Unnamed: 0'],data['PLUS_MINUS'],data['TOTAL']
    del data['FT'],data['FTA']
    del data['OU']
#del data['Team']
#data = pd.get_dummies(data)
    teams = data.Team.unique()
#iterate over those teams, make a rolling window over n games. 
    N_GAMES = window
    nba_data = pd.DataFrame([])
    season_ids = []
    for i,val in enumerate(data['GAME_ID'].values):
        season_ids.append(str(val)[1:3])

    data['Season_ID'] = season_ids

    for team in teams:
       # print(team)
    #get separate seasons here
        team_data = data.loc[data['Team'] == team]  #this contains the box score of every team game from 2013 to 2018.
        for season in data['Season_ID'].unique():
            #print(season)
            team_season = team_data.loc[team_data['Season_ID'] == season]
        
            stuff_to_turn_into_avgs = ['OR', 'DR', 'TOT', 'PF', 'ST', 'TO', 'BL', '3P%', 'FG%', 'FT%','PTS']
            for col in team_season.columns:
                if col in stuff_to_turn_into_avgs:
            #split each season up here, 
                    team_season['Rolling ' + col] = team_season[col].rolling(window=N_GAMES).mean().shift(1)
                    if col != 'PTS':
                        del team_season[col]
            nba_data =  nba_data.append(team_season)

           # df = pd.concat([road_df,home_df],axis=1)
#reorganize the dataset. 
    nba_data_splits = nba_data.sort_values(by = ['GAME_ID', 'Home'], ascending=[True, True])
    nba_data_splits.dropna(inplace=True)  #null values come with rolling means, drop those now. 

#delete columns no longer of use, ie team name etc. Can consider keeping team name and see if helps chances. 
    del nba_data_splits['GAME_ID'],nba_data_splits['Home'],nba_data_splits['Away'],nba_data_splits['Date']
    del nba_data_splits['Team']
#now align the box scores so its one big one for each game, home team and road teams. 

    road_df = nba_data_splits.iloc[::2]
    home_df = nba_data_splits.iloc[1::2]
    for col in nba_data_splits.columns:
        road_df['road_' + col] = road_df[col]
        home_df['home_' + col] = home_df[col]
    
        del road_df[col],home_df[col]

    home_df.reset_index(inplace=True)
    road_df.reset_index(inplace=True)

#merged into a dataframe here. 
    df = pd.concat([road_df,home_df],axis=1)
    del df['index']

#create the dataset here. Can consider the spread, or winner. 
#at the moment only using a single classifier, that seems sufficient. A home team loss is synonymous with a road team win. 

    df['final_SPREAD'] = df['road_PTS'] - df['home_PTS']
    del df['road_PTS'], df['home_PTS'],df['home_SPREAD']
           # if openspread + endspread <0:
            #    y.append(np.array([0,1,0]))  #home team covered
            #elif openspread + endspread >0:
            #    y.append(np.array([1,0,0]))  #road covered
           # else: 
           #     y.append(np.array([0,0,1]))  #push!
    y = []

    if spread: 
        for i in range(len(df)):
            if df['road_SPREAD'].values[i] + df['final_SPREAD'].values[i] < 0:
                y.append(1) #home team covers
            else: # df['road_SPREAD'].values[i] + df['final_SPREAD'].values[i] > 0:
                y.append(0) #road team covers or push
    #else:
    #    y.append(np.array([0,1]))  #push! 
    
    if winner:
        for i in range(len(df)):
            if df['final_SPREAD'].values[i] < 0: #home team won. 
                y.append([0,1])
            else:
                y.append([1,0]) #road team won. 

    del df['final_SPREAD']

#y_names = np.array(['road team win', 'home team win']) #for preprocessing/visualization. 
    X = df
    return X,y

In [4]:
X , y = create_rolling_dataset(window=5,FILENAME='NBADATA.csv',spread=True)

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


KeyboardInterrupt: 

In [ ]:
int(np.linspace(1,9,9))

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

lookback = np.linspace(1,9,9)
for window in lookback:
    print("Testing ",window," day window size.")
    X , y = create_rolling_dataset(window=int(window),FILENAME='NBADATA.csv',spread=True)
    X = X.values

    scaler = MinMaxScaler()
    X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 0)

    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    param_grid = {'C': [.127,.13,.133],'gamma': [.009,.01,.011]}

    grid_search = GridSearchCV(SVC(),param_grid,cv=5)
    X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 0)

    grid_search.fit(X_train,y_train)

    test_score = grid_search.score(X_test,y_test)
    print("Test set score: ", test_score)

    cutoff  = 0.52381

    if test_score>cutoff:
        print("This is profitable!")
        winnings = 1*test_score - 1.1*(1-test_score)
        print("in 10 games w/ 10$ bets, you'd make ", 100*winnings, 'with no outside input')
        #print(grid_search.bestp)

Best windows. 8, 7, 4. These all profit over 1%, so further grid searching on these could lead to best outcome. 



# 4 Day lookback

In [ ]:
window1 = 4

X , y = create_rolling_dataset(window=4,FILENAME='NBADATA.csv',spread=True)
X = X.values


scaler = MinMaxScaler()
X_train,X_test,y_train,y_test = train_test_split(X4,y4,random_state = 0)

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
param_grid = {'C': np.linspace(.1,.15,10), 'gamma': [.007,.008,.0085]}

grid_search = GridSearchCV(SVC(),param_grid,cv=5)

grid_search.fit(X_train,y_train)

test_score = grid_search.score(X_test,y_test)
print("Test set score: ", test_score)

cutoff  = 0.52381

if test_score>cutoff:
    print("This is profitable!")
    winnings = 1*test_score - 1.1*(1-test_score)
    print("in 10 games w/ 10$ bets, you'd make ", 100*winnings, 'with no outside interference')
    print(grid_search.best_params_)

In [ ]:
window2 = 7



# 8 Day lookback

In [ ]:
window3 = 8

X , y = create_rolling_dataset(window=5,FILENAME='NBADATA.csv',spread=True)
X = X.values


scaler = MinMaxScaler()
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 0)

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
param_grid = {'C': [.127,.13,.133],'gamma': [.009,.01,.011]}
grid_search = GridSearchCV(SVC(),param_grid,cv=5)

grid_search.fit(X_train,y_train)

test_score = grid_search.score(X_test,y_test)
print("Test set score: ", test_score)

cutoff  = 0.52381

if test_score>cutoff:
    print("This is profitable!")
    winnings = 1*test_score - 1.1*(1-test_score)
    print("in 10 games w/ 10$ bets, you'd make ", 100*winnings, 'with no outside interference')
    print(grid_search.best_params_)

In [ ]:
print(grid_search.best_params_)

Use a different classifier here. 

In [ ]:
from sklearn.neural_network import MLPClassifier

scaler = MinMaxScaler()
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 0)

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

param_grid={'learning_rate': [.001]}


grid_search = GridSearchCV(MLPClassifier(),param_grid,cv=5)
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 0)

grid_search.fit(X_train,y_train)

test_score = grid_search.score(X_test,y_test)
print("Test set score: ", test_score)

cutoff  = 0.52381

if test_score>cutoff:
    print("This is profitable!")
    winnings = 1*test_score - 1.1*(1-test_score)
    print("in 10 games w/ 10$ bets, you'd make ", 100*winnings, 'with no outside input')

In [ ]:
model = MLPClassifier()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
model.score(X_train,y_train)

In [ ]:
model.score(X_test,y_test)